In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
get_ipython().run_line_magic('matplotlib', 'inline')

In [2]:
#LOADING THE DATA From data.csv
data = pd.read_csv('Breast_Cancer.csv')

In [3]:
# Converting diagnosis to integer - 1 for malignant, 0 - for benign
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
data['diagnosis']=label_encoder.fit_transform(data['diagnosis'])
data['diagnosis'].unique()

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


array([1, 0], dtype=int64)

In [4]:
#Complete data set:
X = data[data.columns[2:32]]
Y = data['diagnosis']
Y = Y.values.reshape(Y.shape[0],1)

In [5]:
#train set assignment (80%):
train_X = X.loc[0:454,X.columns[0:]]
train_Y = Y[0:455]

#test set assignment (20%):
test_X = X.loc[0:143,X.columns[0:]]
test_Y = Y[0:144]

In [6]:
#training set:

mean = train_X.mean()
std_error = train_X.std()
train_X = (train_X - mean)/std_error

#test set:
mean = test_X.mean()
std_error = test_X.std()
test_X = (test_X - mean)/std_error

In [7]:
# Hypothesis i.e. y = a = sigmoid(z) where z = w^TX + b
def sigmoid(z):
    return 1/(1+np.exp(-z))


In [8]:
def random_init(dim):
    w = np.zeros((dim,1))
    b = 0
    return w,b

In [9]:
#hypothesis in logistic regression is y = a = sigmoid(z) = sigmoid(w^TX + b)
def propo(w,b,X,Y):
    
    m = X.shape[0]
    
    #forward propogation
    z = np.dot(X,w) + b
    a = sigmoid(z)
    cost = -np.sum(Y*np.log(a) - (1-Y)*np.log(1-a))/m
    
    
    #backpropogation:
    dz = a-Y
    dw  = np.dot(np.transpose(X),dz)/m
    db = np.sum(dz)/m
    
    grad = {
        "dw":dw,
        "db":db
    }
    return grad,cost

In [10]:
#Gradient descent
def optim(w,b,X,Y,learning_rate,num_iteration):
    costs = []
    
    for i in range(num_iteration):
        grads, cost=propo(w,b,X,Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        #updating w and b
        w  = w - learning_rate*dw
        b  = b - learning_rate*db
          
        if(i%100==0):
            costs.append(cost)
        
    params= {
        "w":w,
        "b":b
    }
    grads = {
        "dw":dw,
        "db":db
    }
    return params,grads,costs

In [11]:
#random init of w,b
w,b = random_init(train_X.shape[1])

#forward, backward & grad. descent:
params,grads,costs = optim(w,b,train_X,train_Y,0.01,2000)

def predict(w,b,X):
    a = sigmoid(np.dot(X,w) + b)
    return a

def oneORzero(x):
    if(x>=0.5):
        return 1
    elif(x<0.5):
        return 0

In [12]:
# Accuracy
temp = predict(params["w"],params["b"],test_X)
test_prediction = np.array(list(map(oneORzero,temp)))
test_prediction = test_prediction.reshape((test_prediction.shape[0],1))
print("Test Accuracy = ",(100 - np.mean(np.abs(test_prediction - test_Y))*100))

Test Accuracy =  92.36111111111111
